## Deployment and Inference
Work with Existing Model Data and Training Jobs
- https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/using_mxnet.html#work-with-existing-model-data-and-training-jobs

In [1]:
%store -r train_byos_job_name

In [2]:
import sagemaker
from sagemaker import get_execution_role
import json
import os
import boto3
import datetime
import logging
import pandas as pd

In [3]:
# Project name
project_name = 'predict-deliver-time'
now = datetime.datetime.now
str_time = now().strftime('%Y-%m-%d-%H-%M-%S-%f')[:-3]
# Region
region = boto3.Session().region_name
# Bucket
bucket = sagemaker.Session().default_bucket() 
# role
role = get_execution_role()
# SageMaker session
sess = sagemaker.Session()
# Resource Type
instance_type = 'ml.c5.9xlarge'

model_name = project_name + '-' + str_time

In [4]:
from sagemaker.mxnet import MXNet
from sagemaker.mxnet import MXNetModel

In [9]:
mx_estimator = MXNet.attach(train_byos_job_name)


2020-11-14 12:03:47 Starting - Preparing the instances for training
2020-11-14 12:03:47 Downloading - Downloading input data
2020-11-14 12:03:47 Training - Training image download completed. Training in progress.
2020-11-14 12:03:47 Uploading - Uploading generated training model
2020-11-14 12:03:47 Completed - Training job completed


In [10]:
predictor = mx_estimator.deploy(instance_type='ml.m4.xlarge',
                                   initial_instance_count=1)

---------------!

In [ ]:
predictor.predict

## Using TAR

In [6]:
## 모델 결과 파일 저장
model_artifact='s3://{}/{}/output/model.tar.gz'.format(bucket, train_byos_job_name)
print('model_artifact: ', model_artifact)                                                                                                              

# training_image, inferene_image 가 동일
inference_image = '763104351884.dkr.ecr.{}.amazonaws.com/mxnet-training:1.6.0-cpu-py3'.format(region)

mxnet_model = MXNetModel(
    # entry_point = 'src/autogluon_inference.py',
    entry_point = 'src/autogluon_train.py',    
    image_uri = inference_image,
    model_data = model_artifact,
    role = role,
    name = model_name
    
)

In [7]:
mxnet_model.entry_point
mxnet_model.name

'predict-deliver-time-2020-11-15-03-08-41-145'

In [8]:
predictor = mxnet_model.deploy(instance_type='ml.m4.xlarge',initial_instance_count=1)

TypeError: expected string or bytes-like object